# 这是一个Jupyter示例
## 这是一个Jupyter示例
### 这是一个Jupyter示例
### 这是一个Jupyter示例
##### 这是一个Jupyter示例
###### 这是一个Jupyter示例


这是一个Jupyter示例



 ## 依赖包的导入
可以使用 import导入整个包，也可以使用from ** import *** 导入某个包里的特定函数或类

In [1]:
# Import Module
from numpy import loadtxt # 从numpy库中导入loadtxt函数
from tensorflow.keras.models import Sequential # 从tensorflow.keras.models模块中导入Sequential类
from tensorflow.keras.layers import Dense

from manas.model2.metadata.model_spec import ModelSpec
from manas.model2.metadata.ParameterBuilder import ParameterBuilder
from manas.model2.metadata.metadataBuilder import MetaDataBuilder
from manas.model2 import repository  as model2

依赖导入时的常见错误

ModuleNotFoundError: No module named ‘ *** ‘

In [8]:
! conda list

/usr/bin/sh: line 1: conda: command not found


## 定义数据集读取函数，这里的示例为文本序列数据。

In [2]:
def read_dataset(data_path):  # data_path: str，数据集文件的路径
    # Data reading
    dataset = loadtxt(data_path, delimiter=',')
    x = dataset[:,0:8] # x: numpy.ndarray，特征
    y = dataset[:,8] # y: numpy.ndarray，标签
    return x, y

## 定义模型训练函数


In [3]:
def train_model(x, y):
    # Train Model
    model = Sequential() # 定义模型为Sequential类模型
    model.add(Dense(12, input_shape=(8,), activation='relu')) # 添加输入层和第一个隐藏层，8个输入节点，12个隐藏层节点，激活函数为ReLU
    model.add(Dense(8, activation='relu')) # 添加第二个隐藏层，8个隐藏层节点，激活函数为ReLU
    model.add(Dense(1, activation='sigmoid')) # 添加输出层，1个节点，激活函数为sigmoid
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # 编译模型，使用binary_crossentropy损失函数，优化器为adam，评估指标为accuracy
    model.fit(x, y, epochs=50, batch_size=10) # 训练模型，迭代50次，每批次10个样本
    return model

## 定义模型评估函数

评估函数为损失函数

评价指标为模型的准确率

In [4]:
def evaluate_model(model, x, y): 
    # Evaluating model
    score = model.evaluate(x, y) # 调用Sequential类自带的evaluate函数
    return score

## 定义模型保存函数

In [5]:
def save_model(model, save_path):
    # Save Model
    model.save(save_path) # 保存模型到指定路径

In [6]:
def register_model(save_path): # 注册模型到模型管理平台
    # Registering Model
    model_spec = ModelSpec.Builder().name("keras_demo").type("tensorflow-pb").model_version("1.0.1").build()
    metadata = MetaDataBuilder().model_spec(model_spec).build()
    param = ParameterBuilder().model(save_path).metadata(metadata).build()
    result = model2.modelRegister(param)
    return result

In [ ]:
print("[MANAS] ---> get train dataset and predict dataset")
local_file = "/dls/app/input/tensorflow_demo_data.csv"
x, y = read_dataset(local_file)

print("[MANAS] ---> train model")
model = train_model(x, y)

print("[MANAS] ---> evaluate model")
result = evaluate_model(model, x, y)
print("evaluate result:", result)

print("[MANAS] ---> do save model")
save_path = "/dls/app/model/keras_demo/"
save_model(model, save_path)

print("[MANAS] ---> register model")
register_model(save_path)

print("[MANAS] ---> Manas training end")